In [26]:
import os
import requests
import pprint
import pandas as pd
import geopandas as gpd
import numpy as np

from covidcaremap.constants import *
from covidcaremap.data import (read_us_states_gdf,
                               read_us_counties_gdf,
                               external_data_path,
                               published_data_path)

In [6]:
AIRTABLE_KEY = os.environ.get('AIRTABLE_KEY')
if AIRTABLE_KEY is None:
    raise Exception('AIRTTABLE_KEY environment variable must be set to run this notebook.')

In [19]:
url = 'https://api.airtable.com/v0/appiZGFluszFy8q2r/County%20Overrides?maxRecords=100&view=Validated&sort%5B0%5D%5Bfield%5D=Source%20Date&sort%5B0%5D%5Bdirection%5D=desc'

headers = {'Authorization': f'Bearer {AIRTABLE_KEY}'}

r = requests.get(url, headers=headers)
records = r.json()['records']

In [53]:
def calc_icu_missing(fields_dict):
    fields_dict_keys = list(fields_dict.keys())
    
    cols = ['Staffed ICU Beds',
            'ICU Beds Currently Available',
            'ICU Beds Currently Occupied',
            ]
    
    # if 2 of the 3 above cols present, calc the missing col, otherwise leave empty cols as nans
    if len(set(fields_dict_keys) & set(cols)) >= 2:
   
        if 'ICU Beds Currently Available' in fields_dict_keys and 'ICU Beds Currently Occupied' in fields_dict_keys:
            fields_dict['Staffed ICU Beds'] = fields_dict['ICU Beds Currently Available'] + fields_dict['ICU Beds Currently Occupied']        
        elif 'ICU Beds Currently Occupied' in fields_dict_keys:
            fields_dict['ICU Beds Currently Available'] = fields_dict['Staffed ICU Beds'] - fields_dict['ICU Beds Currently Occupied']
        else:
            fields_dict['ICU Beds Currently Occupied'] = fields_dict['Staffed ICU Beds'] - fields_dict['ICU Beds Currently Available']

        fields_dict['ICU Bed Occupancy Rate'] = fields_dict['ICU Beds Currently Occupied'] / fields_dict['Staffed ICU Beds']

    else:
        for col in cols:
            if col not in fields_dict_keys: 
                print(f'not enough fields entered to calculate, leaving {col} as nan')
                fields_dict[col] = np.nan
        
    return fields_dict

In [54]:
state_records = []
county_records = []

for rec in records:
    if 'County Name' in rec['fields'].keys(): county_records.append(rec)
    else: state_records.append(rec)

In [55]:
county_records

[{'id': 'recn6K6imMlCiyVha',
  'fields': {'County Name': 'Dane',
   'Staffed ICU Beds': nan,
   'Manual Override New Data Source': 'https://www.nbc15.com/content/news/Area-health-systems-share-ICU-bed-numbers-ahead-of-possible-surge-569494681.html?fbclid=IwAR0pAxD3eBo9vOfQ_gLOVJGS0O5_06xqbb9-YH7ShKJAkk2mCB933LxqBFI',
   'Source Date': '2020-07-10',
   'Data Validation Attachment': [{'id': 'attEOQfsSmI00wopZ',
     'url': 'https://dl.airtable.com/.attachments/c31e671c54394538195e459e0c77b5ae/5c15154e/ScreenShot2020-07-15at2_26_14PM.png',
     'filename': 'Screen Shot 2020-07-15 at 2_26_14 PM.png',
     'size': 89094,
     'type': 'image/png',
     'thumbnails': {'small': {'url': 'https://dl.airtable.com/.attachmentThumbnails/89ac9f4e9adc7f43d268449a6ce111e8/b84c7a4e',
       'width': 27,
       'height': 36},
      'large': {'url': 'https://dl.airtable.com/.attachmentThumbnails/9d1c85cafa8c9b4551ac4e2fcc978e39/e31ed8f2',
       'width': 379,
       'height': 504},
      'full': {'url': 

In [56]:
state_records

[{'id': 'rec7YItHxUuLpAP3B',
  'fields': {'Staffed ICU Beds': 6244,
   'Manual Override New Data Source': 'https://bi.ahca.myflorida.com/t/ABICC/views/Public/ICUBedsHospital?%3AshowAppBanner=false&%3Adisplay_count=n&%3AshowVizHome=n&%3Aorigin=viz_share_link&%3AisGuestRedirectFromVizportal=y&%3Aembed=y',
   'ICU Beds Currently Occupied': 5317,
   'ICU Beds Currently Available': 927,
   'Source Date': '2020-07-08',
   'Data Validation Attachment': [{'id': 'attjkxFESmgT8C6ei',
     'url': 'https://dl.airtable.com/.attachments/b87f17970f9250b053178448783f72a0/bbce4818/ScreenShot2020-07-08at7_27_34PM.png',
     'filename': 'Screen Shot 2020-07-08 at 7_27_34 PM.png',
     'size': 32224,
     'type': 'image/png',
     'thumbnails': {'small': {'url': 'https://dl.airtable.com/.attachmentThumbnails/899a3f7eb2f7839783e4d417539c3c8a/f1b41bf4',
       'width': 122,
       'height': 36},
      'large': {'url': 'https://dl.airtable.com/.attachmentThumbnails/59d806e9d4674062ab637b839c7e2ffe/5e033e98',

In [57]:
state_override_list = []
cols = ['State Name','ICU Bed Occupancy Rate','Staffed ICU Beds','Source Date','Manual Override New Data Source']
    
for rec in state_records:
    try: 
        icu_complete_rec = calc_icu_missing(rec['fields'])
        state_override_list.append({k: v for k, v in icu_complete_rec.items() if k in cols})
    except AssertionError: print(f'not enough ICU fields completed for record {rec["id"]}')
    

state_df = pd.DataFrame.from_dict(state_override_list)
state_df['Source Date'] = pd.to_datetime(state_df['Source Date'])

In [58]:
county_override_list = []
cols = ['State Name','County Name','ICU Bed Occupancy Rate','Staffed ICU Beds','Source Date','Manual Override New Data Source']
    
for rec in county_records:
#     print(rec['fields']['County Name'])
    try: 
        icu_complete_rec = calc_icu_missing(rec['fields'])
        county_override_list.append({k: v for k, v in icu_complete_rec.items() if k in cols})
    except AssertionError: print(f'not enough ICU fields completed for record {rec["id"]}')
    

county_df = pd.DataFrame.from_dict(county_override_list)
county_df['Source Date'] = pd.to_datetime(county_df['Source Date'])

In [59]:
state_df['State'] = state_df['State Name'].apply(lambda x: state_name_to_abbreviation[x])
county_df['State'] = county_df['State Name'].apply(lambda x: state_name_to_abbreviation[x])

In [60]:
state_df_dedup = state_df.sort_values(['Source Date'],ascending=False).drop_duplicates('State')
county_df_dedup = county_df.sort_values(['Source Date'],ascending=False).drop_duplicates(['State', 'County Name'])

In [61]:
state_df

,Staffed ICU Beds,Manual Override New Data Source,Source Date,State Name,ICU Bed Occupancy Rate,State
0,6244,https://bi.ahca.myflorida.com/t/ABICC/views/Pu...,2020-07-08,Florida,0.851537,FL
1,2448,https://hub.mph.in.gov/dataset/covid-19-beds-a...,2020-07-08,Indiana,0.607843,IN
2,2900,"https://www.michigan.gov/coronavirus/0,9753,7-...",2020-07-08,Michigan,0.763793,MI
3,396,https://www.maine.gov/dhhs/mecdc/infectious-di...,2020-07-08,Maine,0.638889,ME
4,2257,https://covid19.ncdhhs.gov/dashboard/hospitali...,2020-07-08,North Carolina,0.796190,NC
5,781,https://govstatus.egov.com/OR-OHA-COVID-19,2020-07-08,Oregon,0.841229,OR
6,169,https://alaska-dhss.maps.arcgis.com/apps/opsda...,2020-07-07,Alaska,0.408284,AK
7,1682,https://azdhs.gov/preparedness/epidemiology-di...,2020-07-07,Arizona,0.913793,AZ
8,345,https://coronavirus.dc.gov/page/hospital-statu...,2020-07-07,District of Columbia,0.765217,DC
9,1875,https://www.mass.gov/doc/covid-19-dashboard-ju...,2020-07-07,Massachusetts,0.466667,MA


In [62]:
state_df_dedup

,Staffed ICU Beds,Manual Override New Data Source,Source Date,State Name,ICU Bed Occupancy Rate,State
0,6244,https://bi.ahca.myflorida.com/t/ABICC/views/Pu...,2020-07-08,Florida,0.851537,FL
2,2900,"https://www.michigan.gov/coronavirus/0,9753,7-...",2020-07-08,Michigan,0.763793,MI
3,396,https://www.maine.gov/dhhs/mecdc/infectious-di...,2020-07-08,Maine,0.638889,ME
4,2257,https://covid19.ncdhhs.gov/dashboard/hospitali...,2020-07-08,North Carolina,0.796190,NC
5,781,https://govstatus.egov.com/OR-OHA-COVID-19,2020-07-08,Oregon,0.841229,OR
1,2448,https://hub.mph.in.gov/dataset/covid-19-beds-a...,2020-07-08,Indiana,0.607843,IN
9,1875,https://www.mass.gov/doc/covid-19-dashboard-ju...,2020-07-07,Massachusetts,0.466667,MA
11,771,https://nebraska.maps.arcgis.com/apps/opsdashb...,2020-07-07,Nebraska,0.539559,NE
10,2198,https://mn.gov/covid19/data/response-prep/resp...,2020-07-07,Minnesota,0.471338,MN
12,1005,https://okomes.cloud.looker.com/embed/dashboar...,2020-07-07,Oklahoma,0.790050,OK


In [63]:
county_df

,County Name,Staffed ICU Beds,Manual Override New Data Source,Source Date,State Name,ICU Bed Occupancy Rate,State
0,Dane,NaN,https://www.nbc15.com/content/news/Area-health...,2020-07-10,Wisconsin,NaN,WI
1,Santa Clara,480.0,https://www.sccgov.org/sites/covid19/Pages/das...,2020-06-23,California,0.539583,CA
2,Santa Clara,480.0,https://www.sccgov.org/sites/covid19/Pages/das...,2020-06-18,California,0.575000,CA
3,Santa Clara,477.0,https://www.sccgov.org/sites/covid19/Pages/das...,2020-06-16,California,0.534591,CA


In [64]:
county_df_dedup

,County Name,Staffed ICU Beds,Manual Override New Data Source,Source Date,State Name,ICU Bed Occupancy Rate,State
0,Dane,NaN,https://www.nbc15.com/content/news/Area-health...,2020-07-10,Wisconsin,NaN,WI
1,Santa Clara,480.0,https://www.sccgov.org/sites/covid19/Pages/das...,2020-06-23,California,0.539583,CA


In [65]:
state_data = pd.read_csv(
    published_data_path('us_healthcare_capacity-state-CovidCareMap.csv'),
    dtype={'State': str }
)

county_data = pd.read_csv(
    published_data_path('us_healthcare_capacity-county-CovidCareMap.csv'),
    dtype={'State': str , 'fips_code': str}
)

In [66]:
county_data.dtypes

fips_code                                      object
State                                          object
County Name                                    object
Staffed All Beds                              float64
Staffed ICU Beds                              float64
Licensed All Beds                             float64
All Bed Occupancy Rate                        float64
ICU Bed Occupancy Rate                        float64
Population                                    float64
Population (20+)                              float64
Population (65+)                              float64
Staffed All Beds [Per 1000 People]            float64
Staffed All Beds [Per 1000 Adults (20+)]      float64
Staffed All Beds [Per 1000 Elderly (65+)]     float64
Staffed ICU Beds [Per 1000 People]            float64
Staffed ICU Beds [Per 1000 Adults (20+)]      float64
Staffed ICU Beds [Per 1000 Elderly (65+)]     float64
Licensed All Beds [Per 1000 People]           float64
Licensed All Beds [Per 1000 

In [67]:
state_df_dedup = pd.merge(state_df_dedup, state_data, on='State', how='left', suffixes=('','_right'))


In [68]:
state_df_dedup.drop(axis=1, 
                    columns=[col for col in state_df_dedup.columns if '_right' in col]
                            +['ICU Bed Source','ICU Bed Source Last Updated'], 
                    inplace=True)

In [69]:
state_df_dedup

,Staffed ICU Beds,Manual Override New Data Source,Source Date,State Name,ICU Bed Occupancy Rate,State,Staffed All Beds,Licensed All Beds,All Bed Occupancy Rate,Population,...,Staffed ICU Beds [Per 1000 People],Staffed ICU Beds [Per 1000 Adults (20+)],Staffed ICU Beds [Per 1000 Elderly (65+)],Licensed All Beds [Per 1000 People],Licensed All Beds [Per 1000 Adults (20+)],Licensed All Beds [Per 1000 Elderly (65+)],Estimated No. Full-Featured Mechanical Ventilators (2010 study estimate),"Estimated No. Full-Featured Mechanical Ventilators per 100,000 Population (2010 study estimate)",Estimated No. Pediatrics-Capable Full-Feature Mechanical Ventilators (2010 study estimate),"Estimated No. Full-Feature Mechanical Ventilators, Pediatrics Capable per 100,000 Population <14 y (2010 study estimate)"
0,6244,https://bi.ahca.myflorida.com/t/ABICC/views/Pu...,2020-07-08,Florida,0.851537,FL,59312.0,76951.0,0.67,21299325,...,0.293,0.376,1.433,3.613,4.640,17.657,4307,23.5,1527,49.6
1,2900,"https://www.michigan.gov/coronavirus/0,9753,7-...",2020-07-08,Michigan,0.763793,MI,22526.0,27031.0,0.61,9995915,...,0.290,0.383,1.689,2.704,3.574,15.747,1847,18.5,660,36.6
2,396,https://www.maine.gov/dhhs/mecdc/infectious-di...,2020-07-08,Maine,0.638889,ME,3073.0,3847.0,0.63,1338404,...,0.296,0.375,1.435,2.874,3.643,13.938,214,16.2,112,54.4
3,2257,https://covid19.ncdhhs.gov/dashboard/hospitali...,2020-07-08,North Carolina,0.796190,NC,23183.0,29396.0,0.64,10383620,...,0.217,0.289,1.336,2.831,3.770,17.402,1782,19.3,901,51.5
4,781,https://govstatus.egov.com/OR-OHA-COVID-19,2020-07-08,Oregon,0.841229,OR,7609.0,9401.0,0.66,4190713,...,0.186,0.243,1.057,2.243,2.921,12.727,503,13.3,216,32.4
5,2448,https://hub.mph.in.gov/dataset/covid-19-beds-a...,2020-07-08,Indiana,0.607843,IN,17211.0,18472.0,0.57,6691878,...,0.366,0.496,2.320,2.760,3.740,17.509,1472,23.1,862,70.5
6,1875,https://www.mass.gov/doc/covid-19-dashboard-ju...,2020-07-07,Massachusetts,0.466667,MA,18936.0,24379.0,0.72,6902149,...,0.272,0.352,1.646,3.532,4.576,21.402,1408,21.7,670,61.7
7,771,https://nebraska.maps.arcgis.com/apps/opsdashb...,2020-07-07,Nebraska,0.539559,NE,5733.0,7482.0,0.53,1929268,...,0.400,0.551,2.539,3.878,5.348,24.639,466,26.1,337,97.0
8,2198,https://mn.gov/covid19/data/response-prep/resp...,2020-07-07,Minnesota,0.471338,MN,11195.0,17693.0,0.60,5611179,...,0.392,0.527,2.470,3.153,4.246,19.884,811,15.5,445,46.1
9,1005,https://okomes.cloud.looker.com/embed/dashboar...,2020-07-07,Oklahoma,0.790050,OK,11767.0,15316.0,0.55,3943079,...,0.255,0.349,1.622,3.884,5.317,24.721,740,20.3,350,49.6


In [70]:
county_df_dedup

,County Name,Staffed ICU Beds,Manual Override New Data Source,Source Date,State Name,ICU Bed Occupancy Rate,State
0,Dane,NaN,https://www.nbc15.com/content/news/Area-health...,2020-07-10,Wisconsin,NaN,WI
1,Santa Clara,480.0,https://www.sccgov.org/sites/covid19/Pages/das...,2020-06-23,California,0.539583,CA


In [71]:
county_df_dedup = pd.merge(county_df_dedup, county_data, on=['State','County Name'], how='left', suffixes=('','_right'))

In [72]:
county_df_dedup.drop(axis=1, 
                     columns=[col for col in county_df_dedup.columns if '_right' in col]
                     +['ICU Bed Source','ICU Bed Source Last Updated'], 
                     inplace=True)

In [73]:
county_df_dedup

,County Name,Staffed ICU Beds,Manual Override New Data Source,Source Date,State Name,ICU Bed Occupancy Rate,State,fips_code,Staffed All Beds,Licensed All Beds,...,Population (65+),Staffed All Beds [Per 1000 People],Staffed All Beds [Per 1000 Adults (20+)],Staffed All Beds [Per 1000 Elderly (65+)],Staffed ICU Beds [Per 1000 People],Staffed ICU Beds [Per 1000 Adults (20+)],Staffed ICU Beds [Per 1000 Elderly (65+)],Licensed All Beds [Per 1000 People],Licensed All Beds [Per 1000 Adults (20+)],Licensed All Beds [Per 1000 Elderly (65+)]
0,Dane,NaN,https://www.nbc15.com/content/news/Area-health...,2020-07-10,Wisconsin,NaN,WI,55025,1734.0,2097.0,...,74433.0,3.197,4.183,23.296,0.072,0.094,0.524,3.866,5.059,28.173
1,Santa Clara,480.0,https://www.sccgov.org/sites/covid19/Pages/das...,2020-06-23,California,0.539583,CA,06085,4252.0,4858.0,...,261131.0,2.195,2.895,16.283,0.248,0.327,1.838,2.507,3.307,18.604


In [74]:
county_df_dedup['Manual Override Reason'] = 'Airtable update flow'
state_df_dedup['Manual Override Reason'] = 'Airtable update flow'

In [75]:
state_df_dedup['Staffed ICU Beds [Per 1000 People]'] = round(state_df_dedup['Staffed ICU Beds'] / (state_df_dedup['Population'] / PER_CAPITA_BASE),3)
state_df_dedup['Staffed ICU Beds [Per 1000 Adults (20+)]'] = round(state_df_dedup['Staffed ICU Beds'] / (state_df_dedup['Population (20+)'] / PER_CAPITA_BASE),3)
state_df_dedup['Staffed ICU Beds [Per 1000 Elderly (65+)]'] = round(state_df_dedup['Staffed ICU Beds'] / (state_df_dedup['Population (65+)'] / PER_CAPITA_BASE),3)

In [76]:
county_df_dedup['Staffed ICU Beds [Per 1000 People]'] = round(county_df_dedup['Staffed ICU Beds'] / (county_df_dedup['Population'] / PER_CAPITA_BASE),3)
county_df_dedup['Staffed ICU Beds [Per 1000 Adults (20+)]'] = round(county_df_dedup['Staffed ICU Beds'] / (county_df_dedup['Population (20+)'] / PER_CAPITA_BASE),3)
county_df_dedup['Staffed ICU Beds [Per 1000 Elderly (65+)]'] = round(county_df_dedup['Staffed ICU Beds'] / (county_df_dedup['Population (65+)'] / PER_CAPITA_BASE),3)

In [77]:
state_df_dedup.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
Staffed ICU Beds,6244,2900,396,2257,781,2448,1875,771,2198,1005,345,1682,169,943,2749
Manual Override New Data Source,https://bi.ahca.myflorida.com/t/ABICC/views/Pu...,"https://www.michigan.gov/coronavirus/0,9753,7-...",https://www.maine.gov/dhhs/mecdc/infectious-di...,https://covid19.ncdhhs.gov/dashboard/hospitali...,https://govstatus.egov.com/OR-OHA-COVID-19,https://hub.mph.in.gov/dataset/covid-19-beds-a...,https://www.mass.gov/doc/covid-19-dashboard-ju...,https://nebraska.maps.arcgis.com/apps/opsdashb...,https://mn.gov/covid19/data/response-prep/resp...,https://okomes.cloud.looker.com/embed/dashboar...,https://coronavirus.dc.gov/page/hospital-statu...,https://azdhs.gov/preparedness/epidemiology-di...,https://alaska-dhss.maps.arcgis.com/apps/opsda...,https://covid19.nj.gov/#live-updates,https://www.mhanet.com/mhaimages/covid-19/dail...
Source Date,2020-07-08 00:00:00,2020-07-08 00:00:00,2020-07-08 00:00:00,2020-07-08 00:00:00,2020-07-08 00:00:00,2020-07-08 00:00:00,2020-07-07 00:00:00,2020-07-07 00:00:00,2020-07-07 00:00:00,2020-07-07 00:00:00,2020-07-07 00:00:00,2020-07-07 00:00:00,2020-07-07 00:00:00,2020-07-06 00:00:00,2020-07-05 00:00:00
State Name,Florida,Michigan,Maine,North Carolina,Oregon,Indiana,Massachusetts,Nebraska,Minnesota,Oklahoma,District of Columbia,Arizona,Alaska,New Jersey,Missouri
ICU Bed Occupancy Rate,0.851537,0.763793,0.638889,0.79619,0.841229,0.607843,0.466667,0.539559,0.471338,0.79005,0.765217,0.913793,0.408284,0.185578,0.568207
State,FL,MI,ME,NC,OR,IN,MA,NE,MN,OK,DC,AZ,AK,NJ,MO
Staffed All Beds,59312,22526,3073,23183,7609,17211,18936,5733,11195,11767,3190,15658,1526,22084,19937
Licensed All Beds,76951,27031,3847,29396,9401,18472,24379,7482,17693,15316,4623,18286,1853,30568,26185
All Bed Occupancy Rate,0.67,0.61,0.63,0.64,0.66,0.57,0.72,0.53,0.6,0.55,0.76,0.6,0.58,0.69,0.62
Population,21299325,9995915,1338404,10383620,4190713,6691878,6902149,1929268,5611179,3943079,702455,7171646,737438,8908520,6126452


In [48]:
county_df_dedup.T

,0,1
County Name,Dane,Santa Clara
Staffed ICU Beds,176,480
Manual Override New Data Source,https://www.nbc15.com/content/news/Area-health...,https://www.sccgov.org/sites/covid19/Pages/das...
Source Date,2020-07-10 00:00:00,2020-06-23 00:00:00
State Name,Wisconsin,California
ICU Bed Occupancy Rate,NaN,0.539583
State,WI,CA
fips_code,55025,06085
Staffed All Beds,1734,4252
Licensed All Beds,2097,4858


In [ ]:
state_df_dedup.to_csv(external_data_path('covidcaremap-ushcsc-state-manual-override.csv'),index=False)
county_df_dedup.to_csv(external_data_path('covidcaremap-ushcsc-county-manual-override.csv'),index=False)